# Multi-Layered Perceptron (MLP) from Scratch

Esse código é um arcabouço básico (feito usando somente Numpy) para criação de uma rede neural simples, composta de múltiplas camadas de Perceptrons.
Tal arcabouço já vem com as seguintes funcionalidades pré-implementadas:
    - funções de ativações básicas (como sigmoid e softmax)
    - um algoritmo de otimização (Stochastic Gradient Descent -- SGD)
    - uma função de perda (cross entropy)
    - processo de treinamento e teste da rede neural

**Seu objetivo é implementar o processo de forward e backpropagation para a camada.
As partes que precisam de implementação estão indicadas ao longo do código com "TODOs".**

### Data Loader

A seção abaixo implementa o carregamento de dois datasets:

    - MNIST, um dataset de dígitos escritos à mão, e
    - SVHN, um dataset de dígitos de imagens do Google Street View

Posteriormente, você poderá escolher qual dataset gostaria de testar sua arquitetura.

In [1]:
import numpy as np
import os
import subprocess
import scipy.io


# load MNIST dataset -- http://yann.lecun.com/exdb/mnist/
def load_mnist(dirpath):
    def download():
        if not os.path.isdir(dirpath):
            os.makedirs(dirpath)

        url_base = 'http://yann.lecun.com/exdb/mnist/'
        for file_name in file_names:
            if os.path.isfile(os.path.join(dirpath, file_name)):
                print('File ' + file_name + ' already exists')
                continue

            url = (url_base + file_name + '.gz').format(**locals())
            print(url)
            out_path = os.path.join(dirpath, file_name + '.gz')
            cmd = ['curl', url, '-o', out_path]
            print('Downloading ', file_name)
            subprocess.call(cmd)
            cmd = ['gzip', '-d', out_path]
            print('Decompressing ', file_name)
            subprocess.call(cmd)

    def load_images(filename):
        """
        Return a 2d array of images from MNIST dataset.
        images : array, shape (n_images, n_pixels)
                 n_pixels = 28*28 = 784
        filename: input data file
        """
        with open(filename, "r") as f:
            magic = np.fromfile(f, dtype=np.dtype('>i4'), count=1)

            n_images = np.fromfile(f, dtype=np.dtype('>i4'), count=1)[0]
            rows = np.fromfile(f, dtype=np.dtype('>i4'), count=1)[0]
            cols = np.fromfile(f, dtype=np.dtype('>i4'), count=1)[0]

            images = np.fromfile(f, dtype=np.ubyte)
            images = images.astype(np.float64) / 255
            images = images.reshape((n_images, rows, cols, 1))

            f.close()

        return images

    def load_labels(filename):
        """
        Return an array of image labels from MNIST dataset.
        labels : array, shape (n_labels)
        filename: input file for labels
        """
        with open(filename, 'r') as f:
            magic = np.fromfile(f, dtype=np.dtype('>i4'), count=1)
            n_labels = np.fromfile(f, dtype=np.dtype('>i4'), count=1)
            labels = np.fromfile(f, dtype=np.uint8)

            f.close()

            return np.squeeze(np.array([one_hot_coding(lbl) for lbl in labels]).astype(np.uint8))

    def one_hot_coding(label):
        if label not in labels_to_categorical:
            y = np.zeros((10, 1), dtype=np.uint8)
            y[label] = 1
            labels_to_categorical[label] = y
        return labels_to_categorical[label]

    labels_to_categorical = dict()

    file_names = ['train-images-idx3-ubyte',
                  'train-labels-idx1-ubyte',
                  't10k-images-idx3-ubyte',
                  't10k-labels-idx1-ubyte']

    download()

    train_data = load_images(os.path.join(dirpath, file_names[0]))
    train_labels = load_labels(os.path.join(dirpath, file_names[1]))
    test_data = load_images(os.path.join(dirpath, file_names[2]))
    test_labels = load_labels(os.path.join(dirpath, file_names[3]))

    return train_data, train_labels, test_data, test_labels


# load SVHN dataset -- http://ufldl.stanford.edu/housenumbers/
def load_svhn(dirpath):
    def download():
        if not os.path.isdir(dirpath):
            os.makedirs(dirpath)

        url_base = 'http://ufldl.stanford.edu/housenumbers/'
        for file_name in file_names:
            if os.path.isfile(os.path.join(dirpath, file_name + '.mat')):
                print('File ' + file_name + ' already exists')
                continue

            url = (url_base + file_name + '.mat').format(**locals())
            print(url)
            out_path = os.path.join(dirpath, file_name + '.mat')
            cmd = ['curl', url, '-o', out_path]
            print('Downloading ', file_name)
            subprocess.call(cmd)

    def load_images(filename):
        """
        Return a 2d array of images from MNIST dataset.
        images : array, shape (n_images, n_pixels)
                 n_pixels = 28*28 = 784
        filename: input data file
        """
        mat = scipy.io.loadmat(filename)
        data = np.rollaxis(mat['X'], -1, 0)
        data = data.astype(np.float64) / 255
        labels = np.squeeze(np.array([one_hot_coding(lbl[0] if lbl[0] != 10 else 0)
                                      for lbl in mat['y']]).astype(np.uint8))

        return data[:, 2:30, 2:30, 0:1], labels  # 28x28 only first band is used

    def one_hot_coding(label):
        if label not in labels_to_categorical:
            y = np.zeros((10, 1), dtype=np.uint8)
            y[label] = 1
            labels_to_categorical[label] = y
        return labels_to_categorical[label]

    labels_to_categorical = dict()

    file_names = ['train_32x32',
                  'test_32x32']

    download()

    train_data, train_labels = load_images(os.path.join(dirpath, file_names[0]))
    test_data, test_labels = load_images(os.path.join(dirpath, file_names[1]))

    return train_data, train_labels, test_data, test_labels

### Funções Básicas

Essa seção de código abaixo implementa as funções de ativação e de custo, além de suas respectivas derivadas.

In [2]:
import numpy as np


# inicializacao dos pesos ####################################################

def glorot_uniform(shape, num_neurons_in, num_neurons_out):
    scale = np.sqrt(6. / (num_neurons_in + num_neurons_out))
    return np.random.uniform(low=-scale, high=scale, size=shape)


def zero(shape):
    return np.zeros(shape)


# ativacoes ################################################################

# sigmoid
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))


# derivative sigmoid
def der_sigmoid(x):
    s = sigmoid(x)
    return s * (1.0 - s)


# softmax
def softmax(x):
    e = np.exp(x - np.amax(x, axis=1, keepdims=True))  # more stable softmax to avoid precision problems
    return e / np.sum(e, axis=1, keepdims=True)


# derivative softmax
def der_softmax(x, y=None):
    s = softmax(x)
    if y is not None:
        k = s[np.where(y == 1)]
        a = - k * s
        a[np.where(y == 1)] = k * (1 - k)
        return a
    return s * (1 - s)


# funcoes objetivos ###########################################################

# cross entropy -- usado no código
def cross_entropy(a, y):
    m = y.shape[0]
    return -np.sum(np.sum(y * np.log(a))) / m


# derivative cross entropy
def der_cross_entropy(a, y):
    m = y.shape[0]
    grad = softmax(a)
    grad[range(m), np.argmax(y, axis=1)] -= 1
    grad = grad/m
    return grad

### Algoritmo de Otimização

Essa seção de código abaixo implementa o algoritmo de otimização, no caso, o SGD.

In [3]:
import abc


class Optimizer:
    __metaclass__ = abc.ABCMeta

    def __init__(self):
        pass

    @abc.abstractmethod
    def apply(self, layers, sum_der_w, sum_der_b, batch_len):
        raise AssertionError


class SGD(Optimizer):
    def __init__(self, lr):
        self.lr = lr

    def apply(self, layers, sum_der_w, sum_der_b, batch_len):
        for i in range(1, len(layers)):
            gw = sum_der_w[layers[i]]/batch_len
            layers[i].w += -(self.lr*gw)

            gb = sum_der_b[layers[i]]/batch_len
            layers[i].b += -(self.lr*gb)

### Camadas

Essa seção de código abaixo implementa as camadas utilizadas para construir as redes neurais.
Nesse caso, há uma classe abstrata (class Layer) que implementa o template que todas as camadas devem seguir.
Além disso, há a classe que implementa a camada que manipula a entrada dos dados (class InputLayer).
Essa camada precisa ser, **obrigatoriamente**, a primeira camada de qualquer rede proposta utilizando esse código, já que ela é responsável por lidas com o dado de entrada.

In [4]:
import abc
import numpy as np


# abstract layer
# essa classe abstrata sera a classe mae de todas as outras camadas
# ela faz com que as outras camadas tenham ao menos duas funcoes (alem da funcao init): feedforward e backpropagation
class Layer:
    __metaclass__ = abc.ABCMeta

    def __init__(self):
        self.in_depth = None
        self.height = None
        self.width = None
        self.out_depth = None
        self.w = None
        self.b = None

    @abc.abstractmethod
    def feedforward(self, prev_layer):
        raise AssertionError

    @abc.abstractmethod
    def backpropagate(self, prev_layer, delta):
        raise AssertionError


# input layer -- this layer MUST be the first one for all proposed networks
class InputLayer(Layer):
    def __init__(self, input_height, input_width, input_channel):
        super(InputLayer, self).__init__()
        self.in_depth = input_channel
        self.height = input_height
        self.width = input_width
        self.out_depth = input_channel
        self.der_act_func = lambda x: x

    def feedforward(self, prev_layer):
        raise AssertionError

    def backpropagate(self, prev_layer, delta):
        raise AssertionError

### Camada Fully Connected

Essa seção de código abaixo implementa uma camada composta de vários perceptrons.
Logo, uma rede neural profunda com várias dessas camadas pode ser vista como uma Multi-Layered Perceptron.

A aula prática hoje será feita essencialmente nesse bloco de código, onde alguns *TODO*s marcam e explicam onde e o que deve ser implementado.
**Em termos gerais, o objetivo da aula é implementar o processo de forward e backpropagation dessa camada.**

**1. Forward**

O processo de forward usa os dados de entrada da camada junto com os pesos e o bias para gerar a saída final.
Tecnicamente, dado uma entrada $a^{l-1}$ da camada $l-1$ anterior, e os pesos $w$ e bias $b$ para uma camada $l$ atual, o forward tem dois passos básicos:

   1. Multiplicação dos pesos e entrada e soma do bias: $z^l = (\sum_i w^l_i*a^{l-1}_i) + b^l$

   2. Atiação via função não linear: $a^l = f(z^l)$

onde $z^l$ é uma variável temporária, $a^l$ é a ativação final da camada, e $f(\cdot)$ é uma função de ativação.

**2. Backpropagation**

O processo de backpropagation recebe o erro provindo da camada posterior e os usa para calcular a derivada dos pesos e bias da camada atual e calcular o erro da camada anterior.
Tecnicamente, suponha que $\delta^l$ seja o erro dessa cama, que vem sendo calculado (via função de custo) e propagado desde a última camada.
Logo, o processo de backpropagation entre as camadas $l$ e $l-1$ pode ser dividido em três passos:

   1. Calcula derivada dos pesos $w^l$: $der^l_w = a^{l-1} * \delta^l$

   2. Calcula derivada do bias $b^l$: $der^l_b = mean(\delta^l)$

   3. Calcula erro $\delta^{l-1}$ para a camada anterior: $\delta^{l-1} = \delta^l * w^l * f'(z^{l-1})$

onde $der_w$ e $der_b$ são variáveis para armazenar as derivadas do peso e do bias respetivamente, $\delta^{l-1}$ é o erro para a camada anterior, e $f'(\cdot)$ é a derivada da função de ativação utilizada.


In [5]:
class PerceptronLayer(Layer):
    def __init__(self, num_inputs, num_outputs, act_func):
        super(PerceptronLayer, self).__init__()
        self.in_depth = num_inputs
        self.height = 1
        self.width = 1
        self.out_depth = num_outputs
        self.act_func = act_func
        self.der_act_func = globals()["der_%s" % act_func.__name__]

        self.w = glorot_uniform((self.in_depth, self.out_depth), self.in_depth, self.out_depth)
        self.b = zero(self.out_depth)

    def feedforward(self, prev_layer):
        """
        Feedforward

        :param prev_layer: a camada anterior
        """
        if prev_layer.a.ndim > 2:
            prev_a = prev_layer.a.reshape((-1, prev_layer.a.shape[1]*prev_layer.a.shape[2]*prev_layer.a.shape[3]))
        else:
            prev_a = prev_layer.a

        self.z = (prev_a.dot(self.w)) + self.b

        self.a = self.act_func(self.z)

        assert self.z.shape == self.a.shape

    def backpropagate(self, prev_layer, delta):
        """
        Backpropagate

        :param prev_layer: a camada anterior no fluxo do backpropagatiom
        :param delta: o erro a ser propagado para a proxima camada
        :returns: a quantidade de alteração dos pesos de entrada dessa camada, a quantidade de alteração dos bias 
        dessa camada, e o erro propagado por essa camada
        """
        assert delta.shape == self.z.shape == self.a.shape

        if prev_layer.a.ndim > 2:
            prev_a = prev_layer.a.reshape((-1, prev_layer.a.shape[1]*prev_layer.a.shape[2]*prev_layer.a.shape[3]))
        else:
            prev_a = prev_layer.a

        der_w = prev_a.T.dot(delta)
        der_b = np.mean(delta, axis=0)

        prev_delta = (delta.dot(self.w.T)).reshape(prev_layer.z.shape) * prev_layer.der_act_func(prev_layer.z)

        return der_w, der_b, prev_delta

### Rede Neural

Essa seção de código abaixo implementa a classe que encapsula o processamento da rede neural.
Especificamente, essa classe realiza todo processamento (tanto forward quanto backpropagation) da rede dado um batch de entrada.
Ela ainda implementa funções auxiliares como para salvar e carregar o modelo.

In [6]:
import pickle
import os


class NeuralNetwork:
    def __init__(self, net, loss):

        assert isinstance(net[0], InputLayer)
        self.input_layer = net[0]

        assert isinstance(net[-1], PerceptronLayer)
        self.output_layer = net[-1]

        self.loss_func = loss
        self.net = net

    def save_model(self, path):
        dict_model = {}
        for i in range(1, len(self.net)):
            dict_model[i, 'w'] = self.net[i].w
            dict_model[i, 'b'] = self.net[i].b
        with open(path, 'wb') as ff:
            pickle.dump(dict_model, ff, pickle.HIGHEST_PROTOCOL)

    def load_model(self, path):
        with open(path, 'rb') as ff:
            dict_model = pickle.load(ff)
            for i in range(1, len(self.net)):
                self.net[i].w = dict_model[i, 'w']
                self.net[i].b = dict_model[i, 'b']

    def feedforward(self, x, y):
        self.input_layer.z = x
        self.input_layer.a = x

        for i in range(len(self.net)-1):
            self.net[i+1].feedforward(self.net[i])

        self.currrent_loss = self.loss_func(self.output_layer.a, y)
        return self.currrent_loss

    def backpropagate(self, optimizer, y):
        sum_der_w = {layer: np.zeros_like(layer.w) for layer in self.net}
        sum_der_b = {layer: np.zeros_like(layer.b) for layer in self.net}

        # propaga o erro
        delta = der_cross_entropy(self.output_layer.z, y)
        for i in range(len(self.net)-1, 0, -1):
            der_w, der_b, prev_delta = self.net[i].backpropagate(self.net[i - 1], delta)
            sum_der_w[self.net[i]] += der_w
            sum_der_b[self.net[i]] += der_b
            delta = prev_delta

        # atualiza os pesos e bias
        optimizer.apply(self.net, sum_der_w, sum_der_b, len(y))

### Main

Essa seção de código abaixo implementa:

    - a rede neural proposta (no caso do exemplo, somente com duas camadas)
    - uma rotina de treino e outro de teste
    - a função main que encapsula tudo
    
Com a arquitetura implementada, usando 100 epochs no dataset todo do MNIST, o resultado obtido foi 89.01% de acurácia na validação em aproximadamente 10 minutos.

In [ ]:
import numpy as np
import datetime
import os
import argparse
import random
import math


def bar(now, end):
    return "[%-10s]" % ("=" * int(10 * now / end))


def mlp():
    input_layer = InputLayer(input_height=28, input_width=28, input_channel=1)
    fc1 = PerceptronLayer(num_inputs=28 * 28, num_outputs=100, act_func=sigmoid)
    fc2 = PerceptronLayer(num_inputs=100, num_outputs=10, act_func=softmax)

    return [input_layer, fc1, fc2]


def training(train_data, train_labels, test_data, test_labels, net, dataset, optimizer, batch_size, output_path, num_epochs):
    start_time = datetime.datetime.now()
    print("Inicio Treino::" + str(start_time.time()))
    for epoch in range(1, num_epochs+1):
        shuffle = np.asarray(random.sample(range(len(train_data)), len(train_data)))

        inputs_done = 0
        for batch in range(0, int(math.ceil(len(train_data) / float(batch_size)))):
            batch_x = train_data[shuffle[batch * batch_size:min(batch * batch_size + batch_size, len(train_data))]]
            batch_y = train_labels[shuffle[batch * batch_size:min(batch * batch_size + batch_size, len(train_data))]]

            batch_loss = net.feedforward(batch_x, batch_y)
            net.backpropagate(optimizer, batch_y)

            inputs_done += min(batch * batch_size + batch_size, len(train_data)) - batch * batch_size
            print("Epoch %02d %s [%d/%d] > Loss: %04f" %
                  (epoch, bar(inputs_done, len(train_data)), inputs_done, len(train_data), batch_loss))

        # save current model
        net.save_model(os.path.join(output_path, dataset + '_model_epoch' + str(epoch) + '.pkl'))
        # test the net at the end of each epoch
        testing(test_data, test_labels, net, batch_size, epoch)
    end_time = datetime.datetime.now()
    print("Final Treino::" + str(end_time.time()))
    print("Tempo de treino: %s segundos" % str((end_time - start_time).total_seconds()))


def testing(test_data, test_labels, net, batch_size, epoch):
    accuracy = 0.0
    for batch in range(0, int(math.ceil(len(test_data) / float(batch_size)))):
        batch_x = test_data[batch * batch_size:min(batch * batch_size + batch_size, len(test_data))]
        batch_y = test_labels[batch * batch_size:min(batch * batch_size + batch_size, len(test_data))]

        net.feedforward(batch_x, batch_y)
        accuracy += sum(np.argmax(net.output_layer.a, axis=1) == np.argmax(batch_y, axis=1))
    accuracy /= float(len(test_data))
    print("Epoch %02d %s [%d/%d] Time %s > Acuracia Validacao: %0.2f%%" %
          (epoch, bar(len(test_data), len(test_data)), len(test_data), len(test_data),
           str(datetime.datetime.now().time()), accuracy * 100))


def feature_extraction(data, labels, net, batch_size, layer):
    features = []
    for batch in range(0, int(math.ceil(len(data) / float(batch_size)))):
        batch_x = data[batch * batch_size:min(batch * batch_size + batch_size, len(data))]
        batch_y = labels[batch * batch_size:min(batch * batch_size + batch_size, len(labels))]

        net.feedforward(batch_x, batch_y)
        features.append(net.net[layer].a)

    return np.asarray(features).reshape(data.shape[0], -1)


def main():
    # general options
    dirpath = os.path.join(os.getcwd(), 'datasets')  # 'Path to the project'
    output_path = os.path.join(os.getcwd(), 'output_folder')  # 'Output path for saving model'
    operation = 'training'  # Operation [options: training | finetuning | feature_extraction | testing]

    # dataset options
    dataset = 'mnist'  # Dataset to be used [options: mnist | svhn]
    subset = False  # Use subset of dataset to train the model (speeds up the training process)

    # deep learning options
    model = None  # Path to load model (required to continue training process and with operations: testing, finetuning and feature_extraction)
    learning_rate = 0.1  # Learning rate for the SGD algorithm
    batch_size = 100  # Batch size
    num_epochs = 100  # Number of epochs

    # initial validations
    if not os.path.isdir(output_path):
        os.mkdir(output_path)

    if (operation == 'feature_extraction' or operation == 'finetuning' or operation == 'testing') \
            and model is None:
        print('Pre-trained model must be provided for operation ', operation)
        raise AssertionError

    # load mnist dataset
    if dataset == 'mnist':
        train_data, train_labels, test_data, test_labels = load_mnist(os.path.join(dirpath, 'mnist'))
    elif dataset == 'svhn':
        train_data, train_labels, test_data, test_labels = load_svhn(os.path.join(dirpath, 'svhn'))
    else:
        print('Dataset not found ', dataset)
        raise NotImplementedError

    # this speeeds up the process
    # should only be used when implementing/debugging
    # should NOT be used when training or performing any other operation
    if subset is True:
        train_data = train_data[:1000, :, :, :]
        train_labels = train_labels[:1000, :]

    net = NeuralNetwork(mlp(), cross_entropy)

    if operation == 'training':
        if model is not None:
            net.load_model(model)
        training(train_data, train_labels, test_data, test_labels, net, dataset,
                 optimizer=SGD(learning_rate),
                 batch_size=batch_size,
                 output_path=output_path,
                 num_epochs=num_epochs)
    elif operation == 'testing':
        net.load_model(model)
        testing(test_data, test_labels, net,
                batch_size=batch_size,
                epoch=1000)
    elif operation == 'finetuning':
        net.load_model(model)
        training(train_data, train_labels, test_data, test_labels, net,
                 optimizer=SGD(learning_rate),
                 batch_size=batch_size,
                 output_path=output_path,
                 num_epochs=num_epochs)
    elif operation == 'feature_extraction':
        net.load_model(model)
        train_features = feature_extraction(train_data, train_labels, net,
                                            batch_size=batch_size,
                                            layer=1)
        test_features = feature_extraction(test_data, test_labels, net,
                                           batch_size=batch_size,
                                           layer=1)
        # features are saved into npy format
        # now, they can be used to train/test a shallow model (svm, random forest, ...)
        np.save(os.path.join(output_path, dataset, '_train_features.npy'), train_features)
        np.save(os.path.join(output_path, dataset, '_test_features.npy'), test_features)
    else:
        print('Operation not found ', operation)
        raise NotImplementedError


if __name__ == "__main__":
    main()


File train-images-idx3-ubyte already exists
File train-labels-idx1-ubyte already exists
File t10k-images-idx3-ubyte already exists
File t10k-labels-idx1-ubyte already exists
Inicio Treino::09:23:13.227906
Epoch 01 [          ] [100/60000] > Loss: 2.388772
Epoch 01 [          ] [200/60000] > Loss: 2.438955
Epoch 01 [          ] [300/60000] > Loss: 2.350196
Epoch 01 [          ] [400/60000] > Loss: 2.392682
Epoch 01 [          ] [500/60000] > Loss: 2.353932
Epoch 01 [          ] [600/60000] > Loss: 2.392939
Epoch 01 [          ] [700/60000] > Loss: 2.380842
Epoch 01 [          ] [800/60000] > Loss: 2.332762
Epoch 01 [          ] [900/60000] > Loss: 2.392153
Epoch 01 [          ] [1000/60000] > Loss: 2.338142
Epoch 01 [          ] [1100/60000] > Loss: 2.298190
Epoch 01 [          ] [1200/60000] > Loss: 2.473802
Epoch 01 [          ] [1300/60000] > Loss: 2.411115
Epoch 01 [          ] [1400/60000] > Loss: 2.410616
Epoch 01 [          ] [1500/60000] > Loss: 2.365486
Epoch 01 [          ] [1

Epoch 01 [==        ] [17400/60000] > Loss: 2.321633
Epoch 01 [==        ] [17500/60000] > Loss: 2.308037
Epoch 01 [==        ] [17600/60000] > Loss: 2.315698
Epoch 01 [==        ] [17700/60000] > Loss: 2.299302
Epoch 01 [==        ] [17800/60000] > Loss: 2.331367
Epoch 01 [==        ] [17900/60000] > Loss: 2.306243
Epoch 01 [===       ] [18000/60000] > Loss: 2.332935
Epoch 01 [===       ] [18100/60000] > Loss: 2.301625
Epoch 01 [===       ] [18200/60000] > Loss: 2.409334
Epoch 01 [===       ] [18300/60000] > Loss: 2.348282
Epoch 01 [===       ] [18400/60000] > Loss: 2.357434
Epoch 01 [===       ] [18500/60000] > Loss: 2.309773
Epoch 01 [===       ] [18600/60000] > Loss: 2.310403
Epoch 01 [===       ] [18700/60000] > Loss: 2.305698
Epoch 01 [===       ] [18800/60000] > Loss: 2.348961
Epoch 01 [===       ] [18900/60000] > Loss: 2.324821
Epoch 01 [===       ] [19000/60000] > Loss: 2.327339
Epoch 01 [===       ] [19100/60000] > Loss: 2.330131
Epoch 01 [===       ] [19200/60000] > Loss: 2.

Epoch 01 [=====     ] [35100/60000] > Loss: 2.246641
Epoch 01 [=====     ] [35200/60000] > Loss: 2.254552
Epoch 01 [=====     ] [35300/60000] > Loss: 2.281052
Epoch 01 [=====     ] [35400/60000] > Loss: 2.285811
Epoch 01 [=====     ] [35500/60000] > Loss: 2.251392
Epoch 01 [=====     ] [35600/60000] > Loss: 2.348604
Epoch 01 [=====     ] [35700/60000] > Loss: 2.232435
Epoch 01 [=====     ] [35800/60000] > Loss: 2.273747
Epoch 01 [=====     ] [35900/60000] > Loss: 2.255762
Epoch 01 [======    ] [36000/60000] > Loss: 2.269213
Epoch 01 [======    ] [36100/60000] > Loss: 2.257315
Epoch 01 [======    ] [36200/60000] > Loss: 2.279517
Epoch 01 [======    ] [36300/60000] > Loss: 2.282176
Epoch 01 [======    ] [36400/60000] > Loss: 2.292893
Epoch 01 [======    ] [36500/60000] > Loss: 2.270625
Epoch 01 [======    ] [36600/60000] > Loss: 2.264862
Epoch 01 [======    ] [36700/60000] > Loss: 2.250786
Epoch 01 [======    ] [36800/60000] > Loss: 2.307767
Epoch 01 [======    ] [36900/60000] > Loss: 2.

Epoch 01 [========  ] [52300/60000] > Loss: 2.264373
Epoch 01 [========  ] [52400/60000] > Loss: 2.263574
Epoch 01 [========  ] [52500/60000] > Loss: 2.261986
Epoch 01 [========  ] [52600/60000] > Loss: 2.219836
Epoch 01 [========  ] [52700/60000] > Loss: 2.243534
Epoch 01 [========  ] [52800/60000] > Loss: 2.246677
Epoch 01 [========  ] [52900/60000] > Loss: 2.258082
Epoch 01 [========  ] [53000/60000] > Loss: 2.282722
Epoch 01 [========  ] [53100/60000] > Loss: 2.231313
Epoch 01 [========  ] [53200/60000] > Loss: 2.260302
Epoch 01 [========  ] [53300/60000] > Loss: 2.300264
Epoch 01 [========  ] [53400/60000] > Loss: 2.238326
Epoch 01 [========  ] [53500/60000] > Loss: 2.214391
Epoch 01 [========  ] [53600/60000] > Loss: 2.272748
Epoch 01 [========  ] [53700/60000] > Loss: 2.225454
Epoch 01 [========  ] [53800/60000] > Loss: 2.253464
Epoch 01 [========  ] [53900/60000] > Loss: 2.261964
Epoch 01 [========= ] [54000/60000] > Loss: 2.221181
Epoch 01 [========= ] [54100/60000] > Loss: 2.

Epoch 02 [=         ] [8900/60000] > Loss: 2.239500
Epoch 02 [=         ] [9000/60000] > Loss: 2.199869
Epoch 02 [=         ] [9100/60000] > Loss: 2.219210
Epoch 02 [=         ] [9200/60000] > Loss: 2.210667
Epoch 02 [=         ] [9300/60000] > Loss: 2.199188
Epoch 02 [=         ] [9400/60000] > Loss: 2.211669
Epoch 02 [=         ] [9500/60000] > Loss: 2.220765
Epoch 02 [=         ] [9600/60000] > Loss: 2.194332
Epoch 02 [=         ] [9700/60000] > Loss: 2.194341
Epoch 02 [=         ] [9800/60000] > Loss: 2.223867
Epoch 02 [=         ] [9900/60000] > Loss: 2.193874
Epoch 02 [=         ] [10000/60000] > Loss: 2.241976
Epoch 02 [=         ] [10100/60000] > Loss: 2.219484
Epoch 02 [=         ] [10200/60000] > Loss: 2.167303
Epoch 02 [=         ] [10300/60000] > Loss: 2.224673
Epoch 02 [=         ] [10400/60000] > Loss: 2.211402
Epoch 02 [=         ] [10500/60000] > Loss: 2.193638
Epoch 02 [=         ] [10600/60000] > Loss: 2.221175
Epoch 02 [=         ] [10700/60000] > Loss: 2.239202
Epoc

...

Epoch 68 [======    ] [38000/60000] > Loss: 0.531555
Epoch 68 [======    ] [38100/60000] > Loss: 0.592988
Epoch 68 [======    ] [38200/60000] > Loss: 0.594256
Epoch 68 [======    ] [38300/60000] > Loss: 0.517479
Epoch 68 [======    ] [38400/60000] > Loss: 0.590287
Epoch 68 [======    ] [38500/60000] > Loss: 0.569340
Epoch 68 [======    ] [38600/60000] > Loss: 0.555901
Epoch 68 [======    ] [38700/60000] > Loss: 0.618217
Epoch 68 [======    ] [38800/60000] > Loss: 0.661809
Epoch 68 [======    ] [38900/60000] > Loss: 0.451807
Epoch 68 [======    ] [39000/60000] > Loss: 0.459760
Epoch 68 [======    ] [39100/60000] > Loss: 0.509758
Epoch 68 [======    ] [39200/60000] > Loss: 0.589221
Epoch 68 [======    ] [39300/60000] > Loss: 0.577757
Epoch 68 [======    ] [39400/60000] > Loss: 0.546420
Epoch 68 [======    ] [39500/60000] > Loss: 0.508603
Epoch 68 [======    ] [39600/60000] > Loss: 0.573771
Epoch 68 [======    ] [39700/60000] > Loss: 0.548133
Epoch 68 [======    ] [39800/60000] > Loss: 0.

Epoch 68 [========= ] [57500/60000] > Loss: 0.554382
Epoch 68 [========= ] [57600/60000] > Loss: 0.491253
Epoch 68 [========= ] [57700/60000] > Loss: 0.607266
Epoch 68 [========= ] [57800/60000] > Loss: 0.601533
Epoch 68 [========= ] [57900/60000] > Loss: 0.601306
Epoch 68 [========= ] [58000/60000] > Loss: 0.565586
Epoch 68 [========= ] [58100/60000] > Loss: 0.541421
Epoch 68 [========= ] [58200/60000] > Loss: 0.580412
Epoch 68 [========= ] [58300/60000] > Loss: 0.657148
Epoch 68 [========= ] [58400/60000] > Loss: 0.487527
Epoch 68 [========= ] [58500/60000] > Loss: 0.547397
Epoch 68 [========= ] [58600/60000] > Loss: 0.555952
Epoch 68 [========= ] [58700/60000] > Loss: 0.621828
Epoch 68 [========= ] [58800/60000] > Loss: 0.493448
Epoch 68 [========= ] [58900/60000] > Loss: 0.525596
Epoch 68 [========= ] [59000/60000] > Loss: 0.609714
Epoch 68 [========= ] [59100/60000] > Loss: 0.589203
Epoch 68 [========= ] [59200/60000] > Loss: 0.671480
Epoch 68 [========= ] [59300/60000] > Loss: 0.

Epoch 69 [==        ] [15100/60000] > Loss: 0.531198
Epoch 69 [==        ] [15200/60000] > Loss: 0.477067
Epoch 69 [==        ] [15300/60000] > Loss: 0.575924
Epoch 69 [==        ] [15400/60000] > Loss: 0.557777
Epoch 69 [==        ] [15500/60000] > Loss: 0.434352
Epoch 69 [==        ] [15600/60000] > Loss: 0.459690
Epoch 69 [==        ] [15700/60000] > Loss: 0.658065
Epoch 69 [==        ] [15800/60000] > Loss: 0.467817
Epoch 69 [==        ] [15900/60000] > Loss: 0.483194
Epoch 69 [==        ] [16000/60000] > Loss: 0.560370
Epoch 69 [==        ] [16100/60000] > Loss: 0.446681
Epoch 69 [==        ] [16200/60000] > Loss: 0.579277
Epoch 69 [==        ] [16300/60000] > Loss: 0.561463
Epoch 69 [==        ] [16400/60000] > Loss: 0.701215
Epoch 69 [==        ] [16500/60000] > Loss: 0.543088
Epoch 69 [==        ] [16600/60000] > Loss: 0.630823
Epoch 69 [==        ] [16700/60000] > Loss: 0.543565
Epoch 69 [==        ] [16800/60000] > Loss: 0.666518
Epoch 69 [==        ] [16900/60000] > Loss: 0.

Epoch 69 [=====     ] [32300/60000] > Loss: 0.632104
Epoch 69 [=====     ] [32400/60000] > Loss: 0.525947
Epoch 69 [=====     ] [32500/60000] > Loss: 0.522738
Epoch 69 [=====     ] [32600/60000] > Loss: 0.644791
Epoch 69 [=====     ] [32700/60000] > Loss: 0.488549
Epoch 69 [=====     ] [32800/60000] > Loss: 0.578537
Epoch 69 [=====     ] [32900/60000] > Loss: 0.491040
Epoch 69 [=====     ] [33000/60000] > Loss: 0.503834
Epoch 69 [=====     ] [33100/60000] > Loss: 0.463717
Epoch 69 [=====     ] [33200/60000] > Loss: 0.522356
Epoch 69 [=====     ] [33300/60000] > Loss: 0.642647
Epoch 69 [=====     ] [33400/60000] > Loss: 0.580477
Epoch 69 [=====     ] [33500/60000] > Loss: 0.666406
Epoch 69 [=====     ] [33600/60000] > Loss: 0.551091
Epoch 69 [=====     ] [33700/60000] > Loss: 0.682230
Epoch 69 [=====     ] [33800/60000] > Loss: 0.614173
Epoch 69 [=====     ] [33900/60000] > Loss: 0.553306
Epoch 69 [=====     ] [34000/60000] > Loss: 0.533492
Epoch 69 [=====     ] [34100/60000] > Loss: 0.

Epoch 69 [========  ] [49400/60000] > Loss: 0.595815
Epoch 69 [========  ] [49500/60000] > Loss: 0.524204
Epoch 69 [========  ] [49600/60000] > Loss: 0.497326
Epoch 69 [========  ] [49700/60000] > Loss: 0.477074
Epoch 69 [========  ] [49800/60000] > Loss: 0.534850
Epoch 69 [========  ] [49900/60000] > Loss: 0.620624
Epoch 69 [========  ] [50000/60000] > Loss: 0.467015
Epoch 69 [========  ] [50100/60000] > Loss: 0.525201
Epoch 69 [========  ] [50200/60000] > Loss: 0.501707
Epoch 69 [========  ] [50300/60000] > Loss: 0.614143
Epoch 69 [========  ] [50400/60000] > Loss: 0.642385
Epoch 69 [========  ] [50500/60000] > Loss: 0.514177
Epoch 69 [========  ] [50600/60000] > Loss: 0.660051
Epoch 69 [========  ] [50700/60000] > Loss: 0.629025
Epoch 69 [========  ] [50800/60000] > Loss: 0.518651
Epoch 69 [========  ] [50900/60000] > Loss: 0.496750
Epoch 69 [========  ] [51000/60000] > Loss: 0.525780
Epoch 69 [========  ] [51100/60000] > Loss: 0.544836
Epoch 69 [========  ] [51200/60000] > Loss: 0.

## Exercícios

1. Altere a quatidade de neurônios das camadas. Isso afeta os resultados?
2. Tente adicionar uma nova camada oculta. Isso afeta os resultados obtidos? E o que dizer sobre o tempo de treinamento?
3. A mudança na taxa de aprendizado (*learning rate*) altera o resultado?
4. Qual é o melhor resultado que você pode obter ao otimizar todos os parâmetros (taxa de aprendizado, iterações, número de camadas ocultas, número de unidades ocultas por camada)?
